In [1]:
import pandas as pd

# Jupyter magic extensions that makes our live easier
%load_ext autoreload
%autoreload 2

In [58]:
from src.models.train_pipeline import train_pipeline
from src.models.models import SklearnAdaBoostModel
from src.data.make_dataset import PandasDatasetLoader

dataset_path = "/media/jmjl/Elements/Epilepsy challenge/1110_train/data/parquet/train/"
labels_path = "/media/jmjl/Elements/Epilepsy challenge"

model, report = train_pipeline(
    PandasDatasetLoader(dataset_path, labels_path), SklearnAdaBoostModel()
)

/home/jmjl/PycharmProjects/epilepsia_challenge/src/data/make_dataset.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return labels
/home/jmjl/PycharmProjects/epilepsia_challenge/src/data/make_dataset.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for i in range(1, 6):
/home/jmjl/PycharmProjects/epilepsia_challenge/src/data/make_dataset.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

n_labels:                                        filepath  label
1318   1110/011/UTC-2020_03_08-10_10_00.parquet      0
1319   1110/011/UTC-2020_03_08-10_20_00.parquet      0
1320   1110/011/UTC-2020_03_08-10_30_00.parquet      0
1321   1110/011/UTC-2020_03_08-10_40_00.parquet      0
1322   1110/011/UTC-2020_03_08-10_50_00.parquet      0
...                                         ...    ...
13887  1110/071/UTC-2020_06_16-23_30_00.parquet      0
16133  1110/080/UTC-2020_07_06-14_10_00.parquet      0
4592   1110/030/UTC-2020_04_04-02_20_00.parquet      0
20214  1110/099/UTC-2020_08_09-19_50_00.parquet      0
19088  1110/093/UTC-2020_07_31-01_00_00.parquet      0

[84 rows x 2 columns]; %_1: 0.42857142857142855
(67, 22)
(17,)
Resultado AUC:  {'1110': 0.8285714285714286} 

Reporte:  {'1110': '              precision    recall  f1-score   support\n\n           0       0.89      0.80      0.84        10\n           1       0.75      0.86      0.80         7\n\n    accuracy                  

In [60]:
print(report["report"]["1110"])

              precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.75      0.86      0.80         7

    accuracy                           0.82        17
   macro avg       0.82      0.83      0.82        17
weighted avg       0.83      0.82      0.82        17



In [61]:
model.model.feature_importances_

array([0.11, 0.05, 0.07, 0.11, 0.02, 0.06, 0.07, 0.04, 0.12, 0.05, 0.04,
       0.  , 0.03, 0.02, 0.06, 0.01, 0.01, 0.03, 0.05, 0.04, 0.01, 0.  ])

In [82]:
from src.features.build_features import FeatExtractor
from src.features.Preprocessor import Preprocessor

g, _ = PandasDatasetLoader(dataset_path, labels_path).get_generators(
    1110,
    feature_extraction_functions=[
        (Preprocessor().convert_time_to_radians, {}),
        (FeatExtractor().mean, {}),
        (FeatExtractor().std, {}),
    ],
)
d, l, _ = next(g)

/home/jmjl/PycharmProjects/epilepsia_challenge/src/data/make_dataset.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels.drop(columns=["patient", "folder"], inplace=True)
/home/jmjl/PycharmProjects/epilepsia_challenge/src/data/make_dataset.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["sum_next_5_labels"] = 0
/home/jmjl/PycharmProjects/epilepsia_challenge/src/data/make_dataset.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [85]:
# d = d.to_frame()
d["importance"] = model.model.feature_importances_
d

,0,importance
utc_timestamp_mean,1.583668e+09,0.11
acc_x_mean,-1.288810e+01,0.05
acc_y_mean,-3.754519e+01,0.07
acc_z_mean,-2.461814e+01,0.11
acc_mag_mean,6.276951e+01,0.02
bvp_mean,2.316680e-02,0.06
eda_mean,2.911571e-01,0.07
hr_mean,5.114656e+01,0.04
temp_mean,3.531520e+01,0.12
time_sin_mean,6.543422e-02,0.05
